In [2]:
import tensorflow as tf 
import os 
from tensorflow import keras 
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from keras import Sequential
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential, load_model

In [5]:
count =0 
dir =os.listdir('train/')
for dir in dir:
    files = list(os.listdir('train/'+dir))
    print(dir + 'folder has '+ str(len(files))+"train")
    count = count + len(files)
print("image folder has "+ str(count) + "image")    

daisyfolder has 501train
dandelionfolder has 646train
rosefolder has 497train
sunflowerfolder has 495train
tulipfolder has 607train
image folder has 2746image


In [7]:
base_dir = ("train/")

In [ ]:
train_dir = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    seed=123,
    image_size = (180, 180),
    subset="training",
    validation_split=0.2,
    batch_size=32  
)

Found 2746 files belonging to 5 classes.
Using 2197 files for training.


In [28]:
val_dir = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    seed=123,
    image_size = (180, 180),
    subset="validation",
    validation_split=0.2,
    batch_size=32 
)

Found 2746 files belonging to 5 classes.
Using 549 files for validation.


In [15]:
flower_name = train_dir.class_names

In [16]:
autotune = tf.data.AUTOTUNE

In [18]:
train_dir = train_dir.cache().shuffle(1000).prefetch(buffer_size = autotune)
val_dir = val_dir.cache().prefetch(buffer_size = autotune)

In [29]:
data_augmentation = Sequential([
    layers.RandomFlip("horizontal", input_shape = (180,180, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

c:\Users\abhis\AppData\Local\anaconda3\envs\brest\lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [30]:
model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    Conv2D(16, 3,padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64 , 3 , padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.2),
    Dense(128, activation="relu"),
    Dense(5)        
])

In [32]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"]
              )

In [33]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_1 (Sequential)       │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 180, 180, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 90, 90, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 90, 90, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 45, 45, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 45, 45, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,965,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,989,285 (15.22 MB)

 Trainable params: 3,989,285 (15.22 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
history = model.fit(train_dir, epochs=15, validation_data=val_dir)

Epoch 1/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 47s 603ms/step - accuracy: 0.3467 - loss: 1.4471 - val_accuracy: 0.5574 - val_loss: 1.1006
Epoch 2/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 38s 545ms/step - accuracy: 0.5834 - loss: 1.0646 - val_accuracy: 0.5956 - val_loss: 1.0423
Epoch 3/15
39/69 ━━━━━━━━━━━━━━━━━━━━ 14s 490ms/step - accuracy: 0.6543 - loss: 0.9311

KeyboardInterrupt: 

In [35]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=3,         
    restore_best_weights=True  
)

history = model.fit(
    train_dir,
    epochs=15,
    validation_data=val_dir,
    callbacks=[early_stopping]  
)


Epoch 1/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 33s 474ms/step - accuracy: 0.6481 - loss: 0.8821 - val_accuracy: 0.6193 - val_loss: 0.9917
Epoch 2/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 35s 510ms/step - accuracy: 0.7036 - loss: 0.7882 - val_accuracy: 0.6266 - val_loss: 0.9684
Epoch 3/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 34s 490ms/step - accuracy: 0.6982 - loss: 0.7792 - val_accuracy: 0.6576 - val_loss: 0.8765
Epoch 4/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 32s 470ms/step - accuracy: 0.7138 - loss: 0.7424 - val_accuracy: 0.6667 - val_loss: 0.8603
Epoch 5/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 34s 490ms/step - accuracy: 0.7165 - loss: 0.7204 - val_accuracy: 0.6794 - val_loss: 0.8504
Epoch 6/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 39s 556ms/step - accuracy: 0.7556 - loss: 0.6401 - val_accuracy: 0.6940 - val_loss: 0.9052
Epoch 7/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 35s 505ms/step - accuracy: 0.7350 - loss: 0.6399 - val_accuracy: 0.7104 - val_loss: 0.8350
Epoch 8/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 33s 484ms/step - accuracy: 0.7751 - loss: 0.5834 - val_accu

In [ ]:
input_image = 